<a href="https://colab.research.google.com/github/vihan-lakshman/mutagenic/blob/main/masking_pipeline_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluation Dataset

In [5]:
!pip install Bio

In [6]:
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#df = pd.read_csv('/content/prelim-deletion-validation-dataset-functional-annotations-with-interpro.csv')

In [7]:
esm3_test_df = pd.read_csv('/content/selectedrcsbpdb.tsv', sep='\t', header = None)

# Drop the last two columns
esm3_test_df = esm3_test_df.iloc[:, :-2]
esm3_test_df.columns = ["Protein Accession", "Seq Digest", "Seq Length", "Analysis", "Signature Accession", "Signature description", "Start location", "Stop location", "Score", "Status", "Date", "InterPro accession", "InterPro description"]

In [8]:
# prompt: read in InterProDescriptions.tsv and for all rows with GO:0016020 in its value for 'GO Terms', add the value for that row's 'Accession' value as part of a growing string separated by a semicolon. Then set it as the variable list_of_membrane_terms. don't name it df. name it interpro_go_df

interpro_go_df = pd.read_csv('/content/InterProDescriptions.tsv', sep='\t')

# Initialize an empty string to store the accessions
list_of_membrane_terms = ""

# Iterate through the DataFrame rows
for index, row in interpro_go_df.iterrows():
    go_terms = row['GO Terms']
    if isinstance(go_terms, str) and "GO:0016020" in go_terms:  # Check if the value is a string and contains "GO:0016020"
        accession = row['Accession']
        if isinstance(accession, str):  # Check if the accession is a string
          if list_of_membrane_terms == "":
            list_of_membrane_terms = accession
          else:
            list_of_membrane_terms += ';' + accession


In [9]:
merged_df = esm3_test_df.groupby('Protein Accession').agg({
    'Seq Length': 'first',
    'InterPro accession': lambda x: ';'.join(x[x != '-'].unique()),
    'InterPro description': lambda x: ';'.join(x[x != '-'].unique())
}).reset_index()

merged_df = merged_df.rename(columns={"InterPro accession": "InterPro_original", "InterPro description": "InterPro_original_description"})

# Add the new column
merged_df['InterPro_target'] = [list_of_membrane_terms] * len(merged_df)
df = merged_df
df

,Protein Accession,Seq Length,InterPro_original,InterPro_original_description,InterPro_target
0,5R1U_1|Chain,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
1,5R1Z_1|Chain,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
2,5R2B_1|Chain,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
3,5R2E_1|Chain,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
4,5R2J_1|Chain,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
...,...,...,...,...,...
231,9FX5_1|Chain,235,IPR001938;IPR037176;IPR017949,Thaumatin family;Osmotin/thaumatin-like superf...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
232,9GI7_1|Chain,129,IPR001916;IPR000974;IPR019799;IPR023346,"Glycoside hydrolase, family 22;Glycoside hydro...",IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
233,9H3E_1|Chain,147,IPR000974;IPR023346;IPR019799;IPR001916,"Glycoside hydrolase, family 22, lysozyme;Lysoz...",IPR000015;IPR000018;IPR000021;IPR000025;IPR000...
234,9HC2_1|Chain,129,IPR001916;IPR000974;IPR019799;IPR023346,"Glycoside hydrolase, family 22;Glycoside hydro...",IPR000015;IPR000018;IPR000021;IPR000025;IPR000...


In [10]:
from Bio import SeqIO
# Read FASTA file into a dictionary
def parse_fasta(fasta_file):
    fasta_dict = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        accession = record.id.split("|")[0]  # Extract only the part before the first "|"
        fasta_dict[accession] = str(record.seq)
    return fasta_dict

# Update DataFrame with sequences
def update_dataframe_with_sequences(df, fasta_dict):
    # Preprocess 'Protein Accession' column
    df["Protein Accession"] = df["Protein Accession"].str.split("|").str[0]
    df["Sequence"] = df["Protein Accession"].map(fasta_dict)
    return df

# Example usage
fasta_file = "/content/selectedrcsbpdb.fasta"
fasta_dict = parse_fasta(fasta_file)
df = update_dataframe_with_sequences(df, fasta_dict)

df

,Protein Accession,Seq Length,InterPro_original,InterPro_original_description,InterPro_target,Sequence
0,5R1U_1,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
1,5R1Z_1,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
2,5R2B_1,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
3,5R2E_1,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
4,5R2J_1,419,IPR034163;IPR021109;IPR001461;IPR001969;IPR033121,Aspergillopepsin-like catalytic domain;Asparti...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MSSPLKNALVTAMLAGGALSSPTKQHVGIPVNASPEVGPGKYSFKQ...
...,...,...,...,...,...,...
231,9FX5_1,235,IPR001938;IPR037176;IPR017949,Thaumatin family;Osmotin/thaumatin-like superf...,IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MAATTCFFFLFPFLLLLTLSRAATFEIVNRCSYTVWAAASKGDAAL...
232,9GI7_1,129,IPR001916;IPR000974;IPR019799;IPR023346,"Glycoside hydrolase, family 22;Glycoside hydro...",IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...
233,9H3E_1,147,IPR000974;IPR023346;IPR019799;IPR001916,"Glycoside hydrolase, family 22, lysozyme;Lysoz...",IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,MRSLLILVLCFLPLAALGKVFGRCELAAAMKRHGLDNYRGYSLGNW...
234,9HC2_1,129,IPR001916;IPR000974;IPR019799;IPR023346,"Glycoside hydrolase, family 22;Glycoside hydro...",IPR000015;IPR000018;IPR000021;IPR000025;IPR000...,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...


In [11]:
# merged_df.to_csv('/content/merged_data.csv', index=False)

# # Download the file (for Google Colab)
# from google.colab import files
# files.download('/content/merged_data.csv')

In [12]:
# import random

# # List of standard amino acid single-letter codes
# amino_acids = "ACDEFGHIKLMNPQRSTVWY"

# def substitute_substring(sequence, del_start, del_end):
#     """Substitutes amino acids in a substring with random amino acids."""

#     # Adjust del_start to be 0-indexed
#     del_start -= 1

#     if del_start < 0 or del_end > len(sequence) or del_start >= del_end:
#         return sequence  # Handle invalid indices

#     substituted_seq = list(sequence)
#     for i in range(del_start, del_end):
#         substituted_seq[i] = random.choice(amino_acids)

#     return "".join(substituted_seq)


# # Apply the function to each row of the DataFrame
# df['substituted_seq'] = df.apply(lambda row: substitute_substring(row['sequence'], row['del_start'], row['del_end']), axis=1)
# df['substituted_seq'].head()

In [13]:
# Dictionary to hold the results
embeddings_dict = {}

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    entry = row['Protein Accession']
    interpro_target = row['InterPro_target']
    interpro_original = row['InterPro_original']

    # Skip rows where 'Interpro' is None
    if pd.isna(interpro_target) or not interpro_target.strip() or pd.isna(interpro_original) or not interpro_target.strip():
        continue

    # Split the InterPro IDs by semicolons
    interpro_ids_target = interpro_target.split(';')
    interpro_ids_target = interpro_ids_target[:-1]
    interpro_ids_original = interpro_original.split(';')
    interpro_ids_original = interpro_ids_original[:-1]

    # Initialize entry in the dictionary if not present
    if entry not in embeddings_dict:
        embeddings_dict[entry] = {
            'InterPro_ids_target': interpro_ids_target,
            'InterPro_ids_original': interpro_ids_original
        }


In [14]:
first_key, first_value = next(iter(embeddings_dict.items()))
print(first_key, first_value['InterPro_ids_original'])

5R1U_1 ['IPR034163', 'IPR021109', 'IPR001461', 'IPR001969']


In [15]:
pip install esm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [16]:
from huggingface_hub import login
from esm.models.esm3 import ESM3
from esm.sdk.api import ESM3InferenceClient, ESMProtein, GenerationConfig

# Will instruct you how to get an API key from huggingface hub, make one with "Read" permission.
login()

In [18]:
model: ESM3InferenceClient = ESM3.from_pretrained("esm3_sm_open_v1").to("cuda")

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

data/esm3_entry.list:   0%|          | 0.00/1.93M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]

data/ParentChildTreeFile.txt:   0%|          | 0.00/595k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

esm3_function_decoder_v0.pth:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

esm3_sm_open_v1.pth:   0%|          | 0.00/2.80G [00:00<?, ?B/s]

esm3_structure_decoder_v0.pth:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

esm3_structure_encoder_v0.pth:   0%|          | 0.00/62.3M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/esm/pretrained.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(


In [19]:
import torch.nn as nn
from esm.tokenization import InterProQuantizedTokenizer
from esm.utils.types import FunctionAnnotation
def get_keywords_from_interpro(
    interpro_annotations,
    interpro2keywords=InterProQuantizedTokenizer().interpro2keywords,
):
    keyword_annotations_list = []
    for interpro_annotation in interpro_annotations:
        keywords = interpro2keywords.get(interpro_annotation.label, [])
        keyword_annotations_list.extend([
            FunctionAnnotation(
                label=keyword,
                start=interpro_annotation.start,
                end=interpro_annotation.end,
            )
            for keyword in keywords
        ])
    return keyword_annotations_list

In [20]:
#protein that only has one function?
#longer sequences of all As, all Gs, or completely random
def get_label_embedding(interpro_label,sequence):
  hostProtein = ESMProtein(sequence=sequence)
  embedding_function = model.encoder.function_embed
  hostProtein.function_annotations = get_keywords_from_interpro([FunctionAnnotation(label=interpro_label, start=1, end=len(sequence))])
  hostProtein_tensor = model.encode(hostProtein)
  device = hostProtein_tensor.function.device  # Get the device of protein2_tensor.function
  embedding_function = embedding_function.to(device)  # Move embedding_function to the device

  function_embed = torch.cat(
      [
          embed_fn(funcs.to(device)) # Ensure funcs is on the same device
          for embed_fn, funcs in zip(
              embedding_function, hostProtein_tensor.function.unbind(-1)
          )
      ],
      -1,
  )

  if function_embed.shape[0] >= 3:
      row_sum = function_embed.sum(dim=0)  # Sum all rows
      row_avg = row_sum / (function_embed.shape[0] - 2)  # Divide by (number of rows - 2)
      row_avg_np = row_avg.cpu().detach().type(torch.float32).numpy()
      return row_avg_np
  else:
      return None

In [21]:
import numpy as np
embedding_target_dict = {}  # Store the average embedding_target for each set of InterPro_ids_target

for entry, interpro_ids in embeddings_dict.items():
    # Check if the embedding_target for this set has already been calculated
    if tuple(interpro_ids['InterPro_ids_target']) not in embedding_target_dict:
        embeddings_target = []
        for interpro_id in interpro_ids['InterPro_ids_target']:
            embeddings_target.append(get_label_embedding(interpro_id, "A"))
        avg_embedding_target = np.mean(embeddings_target, axis=0)
        embedding_target_dict[tuple(interpro_ids['InterPro_ids_target'])] = avg_embedding_target
    else:
        avg_embedding_target = embedding_target_dict[tuple(interpro_ids['InterPro_ids_target'])]

    embeddings_dict[entry]['embedding_target'] = avg_embedding_target

    # Calculate embedding_original as before
    embeddings_original = []
    for interpro_id in interpro_ids['InterPro_ids_original']:
        embeddings_original.append(get_label_embedding(interpro_id, "A"))
    avg_embedding_original = np.mean(embeddings_original, axis=0)
    embeddings_dict[entry]['embedding_original'] = avg_embedding_original

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [22]:
first_key, first_value = next(iter(embeddings_dict.items()))
print(first_key, first_value)

5R1U_1 {'InterPro_ids_target': ['IPR000015', 'IPR000018', 'IPR000021', 'IPR000025', 'IPR000044', 'IPR000045', 'IPR000046', 'IPR000057', 'IPR000060', 'IPR000066', 'IPR000068', 'IPR000072', 'IPR000076', 'IPR000105', 'IPR000109', 'IPR000112', 'IPR000133', 'IPR000136', 'IPR000141', 'IPR000142', 'IPR000144', 'IPR000147', 'IPR000154', 'IPR000155', 'IPR000161', 'IPR000162', 'IPR000163', 'IPR000174', 'IPR000175', 'IPR000189', 'IPR000190', 'IPR000202', 'IPR000204', 'IPR000207', 'IPR000223', 'IPR000248', 'IPR000265', 'IPR000272', 'IPR000276', 'IPR000283', 'IPR000292', 'IPR000293', 'IPR000298', 'IPR000301', 'IPR000314', 'IPR000321', 'IPR000332', 'IPR000333', 'IPR000337', 'IPR000344', 'IPR000351', 'IPR000353', 'IPR000355', 'IPR000356', 'IPR000363', 'IPR000366', 'IPR000369', 'IPR000370', 'IPR000371', 'IPR000374', 'IPR000376', 'IPR000378', 'IPR000382', 'IPR000388', 'IPR000390', 'IPR000393', 'IPR000405', 'IPR000412', 'IPR000425', 'IPR000431', 'IPR000434', 'IPR000452', 'IPR000460', 'IPR000462', 'IPR00

In [23]:
import random

In [24]:
def embedding_masking_model(
    prompt,
    model,
    df,
    embeddings_dict,
    percentage=10,
):
    """
    Helper function to process a protein sequence, calculate similarities,
    and return indices for masking.

    Args:
        prompt (str): The protein sequence to be processed.
        model: The model used for protein generation and embeddings.
        df (pd.DataFrame): DataFrame containing protein data.
        embeddings_dict (dict): Dictionary storing embeddings and other details.

    Returns:
        List[int]: Indices used for masking in the sequence.
    """
    # Create an ESMProtein object
    protein = ESMProtein(sequence=prompt)

    # Configure the model for generation
    generation_config = GenerationConfig(track="function", num_steps=8)

    # Generate the protein
    generated_protein = model.generate(protein, generation_config)

    # Check if function annotations are available
    entry = df.loc[df['Sequence'] == prompt, 'Protein Accession'].iloc[0]
    if generated_protein.function_annotations is None:
        embeddings_dict[entry]['hamming_distance'] = None
        return []

    # Getting embedding for the protein
    protein_tensor = model.encode(generated_protein)
    embedding_function = model.encoder.function_embed
    device = protein_tensor.function.device  # Get the device of protein_tensor.function
    embedding_function = embedding_function.to(device)  # Move embedding_function to the device

    function_embed = torch.cat(
        [
            embed_fn(funcs.to(device))  # Ensure funcs is on the same device
            for embed_fn, funcs in zip(
                embedding_function, protein_tensor.function.unbind(-1)
            )
        ],
        -1,
    )

    # Exclude start and end tokens
    function_embed = function_embed[1:-1, :]

    # Convert the protein_tensor.function to a NumPy array
    protein_np = function_embed.cpu().detach().type(torch.float32).numpy()

    # Retrieve embedding for target function's InterPro IDs (target function = function we're trying to add in addition to, not in place of, original)
    embedding_target = embeddings_dict[entry]['embedding_target']

    # Retrieve embedding for original function's InterPro IDs
    embedding_original = embeddings_dict[entry]['embedding_original']

    # Calculate cosine similarity
    similarities_target = cosine_similarity(protein_np, embedding_target.reshape(1, -1))
    similarities_original = cosine_similarity(protein_np, embedding_original.reshape(1, -1))

    # Get ranks for similarity from highest similarity to lowest similarity
    ranks_target = np.argsort(similarities_target.flatten())[::-1]
    ranks_original = np.argsort(similarities_original.flatten())[::-1]

    #Get number of indices to mask
    num_indices = int(len(prompt) * percentage / 100)

    # Ensure we select at least 1 index
    num_indices = max(1, num_indices)

    # Sum ranks and find top 10 indices with lowest (largest) rank
    ranks = ranks_target + ranks_original
    num_negated = -1*num_indices
    indices = np.argsort(ranks)[num_negated:]

    # Store the indices in the embeddings_dict
    embeddings_dict[entry]['indices'] = indices.tolist()

    return indices.tolist()

In [25]:
def get_random_indices(prompt, percentage):
    """
    Randomly select indices to mask based on the percentage of the prompt's length.
    """
    num_indices = int(len(prompt) * percentage / 100)
    # Ensure we select at least one index
    num_indices = max(1, num_indices)

    # Randomly select unique indices to mask
    return random.sample(range(len(prompt)), num_indices)

In [26]:
for idx, prompt in enumerate(df['Sequence']):
  try:
    random_indices = get_random_indices(prompt, 8)
    # Replace locations in prompt corresponding to indices in 'indices' with "_"
    target_seq = df.loc[df['Sequence'] == prompt, 'Sequence'].iloc[0]
    entry = df.loc[df['Sequence'] == prompt, 'Protein Accession'].iloc[0]
    if entry not in embeddings_dict:
        continue
    for method in ['embedding', 'random']:
      if method == 'embedding':
        indices = embedding_masking_model(prompt, model, df, embeddings_dict,percentage=10)
        if not indices:
          continue
      elif method == 'random':
        continue
        # indices = get_random_indices(prompt, 8)
      modified_prompt = list(prompt)
      for index in indices:
          modified_prompt[index] = "_"
      modified_prompt = "".join(modified_prompt)
      protein_prompt = ESMProtein(sequence=modified_prompt)
      sequence_generation = model.generate(
          protein_prompt,
          GenerationConfig(
              track="sequence",
              num_steps=protein_prompt.sequence.count("_") // 2,
              temperature=0.5,
          ),
      )
      # print("Generated Sequence: " + str(sequence_generation.sequence))
      generated_sequence = sequence_generation.sequence
      # print("Target Sequence: " + str(target_seq))
      # Ensure sequences are of equal length
      if len(generated_sequence) != len(target_seq):
          raise ValueError("Sequences must be of the same length to calculate Hamming distance.")

      # Calculate Hamming distance
      hamming_distance = sum(1 for gen, target in zip(generated_sequence, target_seq) if gen != target)

      # Print results
      print(f'Index {idx}: {method} Hamming Distance:', hamming_distance)
      embeddings_dict[entry][f'{method}_hamming_distance'] = hamming_distance
      embeddings_dict[entry][f'{method}_edited_sequence'] = generated_sequence
      torch.cuda.empty_cache()
  except Exception as e:
    print(f"An error occurred for index {idx}: {e}")
    continue

100%|██████████| 8/8 [00:11<00:00,  1.38s/it]
/usr/local/lib/python3.11/dist-packages/esm/pretrained.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.

Index 0: embedding Hamming Distance: 22


100%|██████████| 20/20 [00:11<00:00,  1.68it/s]


Index 1: embedding Hamming Distance: 24


100%|██████████| 20/20 [00:12<00:00,  1.64it/s]


...

Index 31: embedding Hamming Distance: 22


100%|██████████| 20/20 [00:13<00:00,  1.53it/s]


Index 32: embedding Hamming Distance: 25


 95%|█████████▌| 19/20 [00:12<00:00,  1.49it/s]


KeyboardInterrupt: 

In [ ]:
# prompt: go through embeddings_dict and turn it into a dataframe where the key becomes the first column and each key value pair in the value dict becomes subsequent columns

df_embeddings = pd.DataFrame.from_dict(embeddings_dict, orient='index')

# Reset the index to make the keys a regular column
df_embeddings = df_embeddings.reset_index()

# Rename the 'index' column to 'Protein Accession'
df_embeddings = df_embeddings.rename(columns={'index': 'Protein Accession'})

df_embeddings


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract Hamming distances for 'embedding' and 'random' methods
embedding_distances = []
random_distances = []

# Iterate over embeddings_dict to gather the hamming distances
for key, entry in embeddings_dict.items():
    embedding_distance = entry.get('embedding_hamming_distance', None)
    random_distance = entry.get('random_hamming_distance', None)
    seq_length = len(df.loc[df['Protein Accession'] == key, 'Sequence'].iloc[0])
    embeddings_dict[key]['seq_length'] = seq_length

    # Skip None values and collect valid distances (only compare on proteins w/ both values)
    if embedding_distance is not None and random_distance is not None:
        embedding_distances.append(embedding_distance/seq_length)
        random_distances.append(random_distance/seq_length)

# Calculate average and standard deviation for both methods
embedding_avg = np.mean(embedding_distances) if embedding_distances else 0
embedding_std = np.std(embedding_distances) if embedding_distances else 0
print(embedding_avg)
print(embedding_std)

random_avg = np.mean(random_distances) if random_distances else 0
random_std = np.std(random_distances) if random_distances else 0
print(random_avg)
print(random_std)

# Prepare data for the bar plot
methods = ['Embedding', 'Random']
averages = [embedding_avg, random_avg]
std_devs = [embedding_std, random_std]

# Create the bar plot
fig, ax = plt.subplots()

# Set positions for the bars
bar_width = 0.35
index = np.arange(len(methods))

# Plot the bars with error bars
ax.bar(index, averages, bar_width, yerr=std_devs, capsize=5, label='Average Hamming Distance')

# Customize the plot
ax.set_xlabel('Method')
ax.set_ylabel('Hamming Distance')
ax.set_title('Average Hamming Distance and Standard Deviation by Method')
ax.set_xticks(index)
ax.set_xticklabels(methods)
ax.legend()

# Display the plot
plt.show()

In [ ]:
import json
import numpy as np

# Specify the file path where you want to save the JSON file
file_path = 'hamming_distance_dict.json'

# Define a custom encoder to handle NumPy arrays
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert NumPy array to list
        return json.JSONEncoder.default(self, obj)

# Save the dictionary to a JSON file using the custom encoder
with open(file_path, 'w') as json_file:
    json.dump(embeddings_dict, json_file, indent=4, cls=NumpyEncoder)
    # Use cls=NumpyEncoder to apply the custom encoding